# WES|WGSPipeline 

In [3]:
import os
import sys

#s3 address of input files and output files
s3_input_files_address = "s3://ucsd-ccbb-interns/Mustafa/wgs_test/Sample_cDNA/"
s3_output_files_address = "s3://ucsd-ccbb-interns/Mustafa/wgs_test/Sample_cDNA/gatk_run"

## CFNCluster name
your_cluster_name = "mustafa7"

## The private key pair for accessing cluster.
private_key = "/home/mustafa/interns_oregon_key.pem"

## Project information
project_name = "bwa_gatk_run"

#bwa_gatk or bwa_mutect
workflow = "bwa_gatk"

#hg19 or GRCm38
genome = "hg19"

## If delete cfncluster after job is done.
delete_cfncluster= False

print("variables set")

variables set


In [4]:
sys.path.append("../../src/cirrus_ngs")
from cfnCluster import CFNClusterManager, ConnectionManager

## Create a new cluster
master_ip_address = CFNClusterManager.create_cfn_cluster(cluster_name=your_cluster_name)
ssh_client = ConnectionManager.connect_master(hostname=master_ip_address,
               username="ec2-user",
               private_key_file=private_key)

cluster mustafa7 does exist.

Status: CREATE_COMPLETE
Status: CREATE_COMPLETE
MasterServer: RUNNING
MasterServer: RUNNING
Output:"MasterPublicIP"="34.214.180.149"
Output:"MasterPrivateIP"="172.31.33.31"
Output:"GangliaPublicURL"="http://34.214.180.149/ganglia/"
Output:"GangliaPrivateURL"="http://172.31.33.31/ganglia/"

connecting
connected


In [11]:
from util import PipelineManager
from util import DesignFileLoader

## Possible analysis_steps inputs for the two workflows. Order of input does not matter.

##bwa_gatk: "fastqc", "trim", "bwa", "sort", "dedup", "split", "postalignment", 
#"haplotype", "merge", "combine_vcf", "filter"

##bwa_mutect: "fastqc", "trim" , "bwa", "multiqc", "sort", "dedup", "split", "mutect", "bam_merge", "pair_vcf_merge"
analysis_steps = {
#                     "fastqc"
#                     ,"trim"
#                     ,"bwa"
#                     ,"multiqc"
#                     ,"sort"
#                     ,"dedup"
#                     ,"split"
#                     ,"postalignment"
#                     ,"haplotype"
                    #,"mutect"
                    "merge" 
                    #,"bam_merge"
                    #,"pair_vcf_merge"
                    ,"combine_vcf"
                    #"filter"
                }

#add design file path here
#examples in cirrus_root/data/cirrus-ngs/
design_file = "/home/mustafa/ccbb/cirrus-ngs/data/cirrus-ngs/dnaseq_design_example.txt"

sample_list, group_list, pairs_list = DesignFileLoader.load_design_file(design_file)

PipelineManager.execute("DNASeq", ssh_client, project_name, workflow, analysis_steps, s3_input_files_address,
                       sample_list, group_list, s3_output_files_address, genome, "NA", pairs_list)

[['Sample_cDNA93_R1.fq.gz', 'Sample_cDNA93_R2.fq.gz'], ['Sample_cDNA94_R1.fq.gz', 'Sample_cDNA94_R2.fq.gz']]
['groupA', 'groupA']
{}
making the yaml file...
copying yaml file to remote master node...
bwa_gatk_run.yaml
/shared/workspace/Pipelines/yaml_files/DNASeq/bwa_gatk
executing pipeline...
Executing nohup bash /shared/workspace/Pipelines/scripts/run.sh /shared/workspace/Pipelines/yaml_files/DNASeq/bwa_gatk/bwa_gatk_run.yaml /shared/workspace/logs/DNASeq/bwa_gatk/bwa_gatk_run DNASeq_bwa_gatk
